In [2]:
# Data manipulation
import pandas as pd
import numpy as np

# Data Viz
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# More Data Preprocessing & Machine Learning
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore') 

## Import parcels data

In [ ]:
df0 = pd.read_csv('https://dev.km2.ai/public/parcels_last.csv', low_memory=False)

In [ ]:
df0.shape

(2485732, 55)

In [ ]:
# initial cleaning
df = df0[df0['YearBuilt']>=1900]
df = df[df['GeneralUseType']=='Residential']
df = df[df['zip2'].notna()]
df = df[df['TotalValue']>0]
df = df[df['SQFTmain']>0]

In [ ]:
# calculate price per square foot for properties which have non-zero prices and sizes
df['price_sf'] = df['TotalValue'] / df['SQFTmain']

In [ ]:
df = df.drop(['geo','Location1','ParcelBoundaryDescription','UnitNo','StreetName','HouseNo',
              'HouseFraction','StreetDirection','SpecificUseDetail1','SpecificUseDetail2'], axis = 1)

In [ ]:
df = df.drop(['SpecialParcelClassification','ZIPcode5','TaxRateArea_CITY','TaxRateArea',
              'PropertyUseCode','rowID','GeneralUseType','ZIPcode'], axis = 1)

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.shape

(2053780, 38)

## Calculate distance

In [ ]:
# TO DO: calculate distance to UCLA
ucla = {'lat':34.069444, 'lon':-118.445278}

from geopy.distance import geodesic
def calc_distance_to_ucla(row):
    p1 = (ucla.get('lat'), ucla.get('lon'))
    p2 = (row['CENTER_LAT'], row['CENTER_LON'])
    return geodesic(p1, p2).miles

df['distance_to_ucla'] = df.apply(calc_distance_to_ucla, axis=1)

In [ ]:
# TO DO: calculate distance to USC
usc = {'lat':34.0223519, 'lon':-118.285117}

def calc_distance_to_usc(row):
    p1 = (usc.get('lat'), usc.get('lon'))
    p2 = (row['CENTER_LAT'], row['CENTER_LON'])
    return geodesic(p1, p2).miles

df['distance_to_usc'] = df.apply(calc_distance_to_usc, axis=1)

In [ ]:
# TO DO: create school distrit datasets
df_usc = df[df['distance_to_usc']<3]
df_ucla = df[df['distance_to_ucla']<3]

In [ ]:
df_ucla.to_csv('parcels_ucla.csv')
df_usc.to_csv('parcels_usc.csv')

## Wrangle the school-based data

In [3]:
df_ucla = pd.read_csv('/content/drive/MyDrive/econ445/parcels_ucla.csv', low_memory=False)
zillow_ucla = pd.read_csv('/content/drive/MyDrive/econ445/zillow_ucla.csv', low_memory=False)

In [4]:
df_usc = pd.read_csv('/content/drive/MyDrive/econ445/parcels_usc.csv', low_memory=False)
zillow_usc = pd.read_csv('/content/drive/MyDrive/econ445/zillow_usc.csv', low_memory=False)

In [6]:
#TO DO: filter roll year for proper merge
df_ucla = df_ucla[df_ucla['RollYear']>=2013]
df_usc = df_usc[df_usc['RollYear']>=2013]

In [8]:
zillow_ucla.head()

,zip2,RollYear,ZHVI
0,90017,2021,684000
1,90022,2021,529000
2,90024,2013,908000
3,90024,2014,1080000
4,90024,2015,1100000


In [10]:
#TO DO: merge parcels data and zillow data by zip code and roll year
df_ucla_new = pd.merge(df_ucla, zillow_ucla, on=['zip2','RollYear'])
df_ucla_new = df_ucla_new.drop('Unnamed: 0', axis=1)

In [12]:
df_usc_new = pd.merge(df_usc, zillow_usc, on=['zip2','RollYear'])
df_usc_new = df_usc_new.drop('Unnamed: 0', axis=1)

In [13]:
# TO DO: merge two schools data by row
df = pd.concat([df_ucla_new,df_usc_new], axis=0)

In [15]:
df.head()

,id,AIN,RollYear,AssessorID,PropertyLocation,PropertyType,SpecificUseType,totBuildingDataLines,YearBuilt,EffectiveYearBuilt,SQFTmain,Bedrooms,Bathrooms,Units,RecordingDate,LandValue,LandBaseYear,ImprovementValue,ImpBaseYear,TotalLandImpValue,HomeownersExemption,RealEstateExemption,FixtureValue,FixtureExemption,PersonalPropertyValue,PersonalPropertyExemption,isTaxableParcel,TotalValue,TotalExemption,netTaxableValue,AdministrativeRegion,Cluster,City,CENTER_LAT,CENTER_LON,RecordingDateYear,zip2,price_sf,distance_to_ucla,distance_to_usc,ZHVI
0,5886183,5001026010,2020,5001-026-010,5310 S HOOVER ST LOS ANGELES CA 90037,SFR,Single Family Residence,1,1914,1914,1088,2,2,1,20140918,164838.0,2015,54945.0,2015,219783.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,219783.0,0.0,219783.0,9,9110,LOS ANGELES CA,33.993784,-118.286727,2014,90037.0,202.006434,10.486841,1.971209,521000
1,5898370,5018026017,2020,5018-026-017,700 W 47TH ST LOS ANGELES CA 90037,R-I,Four Units (Any Combination),2,2019,2019,6041,19,12,4,20190513,1173000.0,2019,1051620.0,2019,2224620.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,2224620.0,0.0,2224620.0,9,9401,LOS ANGELES CA,34.000340,-118.285650,2019,90037.0,368.253600,10.324066,1.517481,521000
2,5900340,5021011018,2020,5021-011-018,4238 S HARVARD BLVD LOS ANGELES CA 90062,R-I,"Double, Duplex, or Two Units",2,1913,1927,2085,3,3,2,20050428,30303.0,1976,25471.0,1976,55774.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,55774.0,0.0,55774.0,9,9401,LOS ANGELES CA,34.006229,-118.306436,2005,90062.0,26.750120,9.080358,1.652903,600000
3,5911752,5042028005,2020,5042-028-005,1816 W 35TH PL LOS ANGELES CA 90018,SFR,Single Family Residence,1,1923,1923,832,2,1,1,20081113,24688.0,1978,12438.0,1978,37126.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,37126.0,0.0,37126.0,9,9172,LOS ANGELES CA,34.023397,-118.312577,2008,90018.0,44.622596,8.248579,1.577596,767000
4,5914759,5042028006,2020,5042-028-006,1814 W 35TH PL LOS ANGELES CA 90018,SFR,Single Family Residence,1,1923,1923,832,2,1,1,20081113,17011.0,1975,8263.0,1975,25274.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,25274.0,0.0,25274.0,9,9172,LOS ANGELES CA,34.023397,-118.312465,2008,90018.0,30.377404,8.254516,1.571165,767000


In [19]:
df.to_csv('properties.csv',index=False)

In [16]:
# df_ucla[df_ucla.PropertyLocation.str.find('740 T') != -1]

In [ ]:
X_train = df_ucla[['PropertyType', 'totBuildingDataLines', 'YearBuilt',
       'EffectiveYearBuilt', 'SQFTmain', 'Bedrooms', 'Bathrooms', 'Units',
       'LandValue', 'LandBaseYear', 'ImprovementValue',
       'ImpBaseYear', 'TotalLandImpValue', 'HomeownersExemption',
       'RealEstateExemption', 'FixtureValue', 'FixtureExemption',
       'PersonalPropertyValue', 'PersonalPropertyExemption', 'isTaxableParcel',
       'TotalValue', 'TotalExemption', 'netTaxableValue',
       'AdministrativeRegion',
       'RecordingDateYear', 'zip2','distance_to_ucla',
       'distance_to_usc']]

Y_train = df_ucla['price_sf']